# SAM 3 Zero-Shot Object Detection on DVARF Dataset

This notebook demonstrates the zero-shot object detection capabilities of Meta's Segment Anything Model 3 (SAM 3) on the DVARF dataset for accident scene analysis from drone imagery.

## 1. Environment Setup

Clone the repository and navigate to project directory.

In [ ]:
%cd /content

# Clone the DVARF repository
!git clone https://github.com/AntoFratta/DVARF.git

%cd /content/DVARF

## 2. Install Dependencies

Install project dependencies. Note that we filter out Windows-specific packages (e.g., `triton-windows`) which are not compatible with Linux-based Colab environment.

In [ ]:
# Create a Colab-compatible requirements file by removing Windows-specific packages
!grep -v "triton-windows" requirements.txt > requirements_colab.txt

# Install all dependencies from the filtered requirements file
!pip install -r requirements_colab.txt

# Install triton for Linux
!pip install triton

## 3. Extract Dataset

Extract the data archive which contains images and YOLO-format annotations for train, validation, and test splits.

In [ ]:
%cd /content/DVARF

# Install unrar utility (required only once)
!apt-get update -y > /dev/null
!apt-get install -y unrar > /dev/null

# Extract data.rar which creates the data/ directory structure
!unrar x data.rar ./ > /dev/null

# Verify the extraction
print("After extracting data.rar:")
!ls
print("\nInside data/:")
!ls data
print("\nInside data/raw/:")
!ls data/raw
print("\nImage splits:")
!ls data/raw/images

## 4. Install SAM 3

Clone and install the official SAM 3 repository from Meta.

In [ ]:
%cd /content

# Clone the official SAM 3 repository
!git clone https://github.com/facebookresearch/sam3.git

In [ ]:
%cd /content/sam3

# Install SAM 3 in editable mode
!pip install -e .

## 5. Hugging Face Authentication

Authenticate with Hugging Face to access the SAM 3 model weights.

In [ ]:
from huggingface_hub import login

# Login to Hugging Face (a widget will appear to enter your token)
login()

## 6. Run SAM 3 on Test Split

Execute zero-shot inference using SAM 3 on the test set.

In [ ]:
%cd /content/DVARF

import sys
if "/content/DVARF" not in sys.path:
    sys.path.insert(0, "/content/DVARF")

from scripts.run_sam3_on_split import run_sam3_on_split

# Run SAM 3 zero-shot inference on the test split
# Set save_segmentations=True to save segmentation masks for visualization
run_sam3_on_split(
    split="test",
    score_threshold=0.26,
    max_images=None,
    save_segmentations=True,
    max_masks_per_image_per_class=None,
)

## 7. Evaluate Results

Compute precision, recall, F1-score, and mean Average Precision (mAP) on the test set.

In [ ]:
%cd /content/DVARF

# Evaluate SAM 3 predictions and save metrics to results folder
!python scripts/eval_sam3_on_split.py | tee results/sam3_test_metrics.txt